In [ ]:
%run 'table_ui.ipynb'
%run 'viz_ui.ipynb'
%run 'export_ui.ipynb'

In [ ]:
test_tile

In [ ]:
file_tile

In [ ]:
tb_map_tile

In [ ]:
viz_input_tile

In [ ]:
export_tile

In [ ]:
export_results